In [65]:
from langchain.llms import GooglePalm
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')

In [66]:
llm = GooglePalm(google_api_key= api_key, temperature=0.7)

In [67]:
print(llm('how to make vada paav with time calculation'))

**Ingredients**

* 1 cup urad dal (black gram)
* 1/2 cup rice
* 1 green chili, chopped
* 1/2 teaspoon ginger-garlic paste
* 1/2 teaspoon cumin seeds
* 1/4 teaspoon turmeric powder
* 1/2 teaspoon red chili powder
* 1/4 teaspoon garam masala powder
* 1/2 teaspoon salt
* Oil for deep frying
* 4 bread buns, split
* 1/2 cup green chutney
* 1/2 cup tamarind chutney
* 1/4 cup finely chopped onions
* 1/4 cup finely chopped tomatoes
* 1/4 cup cilantro leaves, for garnish

**Instructions**

1. Soak the urad dal and rice in water for 4-5 hours.
2. Drain the water and grind the dal and rice together to a smooth paste. Add the green chili, ginger-garlic paste, cumin seeds, turmeric powder, red chili powder, garam masala powder, and salt. Mix well to combine.
3. Heat oil in a deep frying pan. Drop spoonfuls of the batter into the hot oil and fry until golden brown. Drain on paper towels.
4. To assemble the vada pav, split the buns in half. Spread some green chutney on the bottom half of each bun. Ad

## Now we are connecting with our database from MySQL

In [68]:
from langchain.utilities import SQLDatabase

In [69]:
db_user ='root'
db_password ='root'
db_host ='localhost'
db_name ='atliq_tshirts'

In [70]:
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)

In [71]:
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [72]:
from langchain_experimental.sql import SQLDatabaseChain

In [73]:
db_chain = SQLDatabaseChain(llm = llm, database = db, verbose = True)

C:\ML Projects\LLM - Projects\SQL_LLM\sql_llm\Lib\site-packages\langchain_experimental\sql\base.py:75: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [74]:
q1 = db_chain.run('how many nike tshirts in small size')



> Entering new SQLDatabaseChain chain...
how many nike tshirts in small size
SQLQuery:SELECT count(*) FROM t_shirts WHERE brand='Nike' AND size='S'
SQLResult: [(3,)]
Answer:3
> Finished chain.


In [75]:
q1

'3'

In [76]:
q2 = db_chain.run('how much is the price of inventory for all small size tshirts?')



> Entering new SQLDatabaseChain chain...
how much is the price of inventory for all small size tshirts?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('311'),)]
Answer:311
> Finished chain.


in the above example we could have like wrong answer i dont how did it manage to give correct answer bt we will giver it few more  defilcult so we will decide how we can improve it 

In [77]:
# q3 = db_chain.run('if we have to sell all the levis tshorts today with discount applied how much our store will genrate')

see in this question is dificult to address like its complex query and many it is using its own genrel knowledge like usually 
wehen get this question there should be coulumns like start date of discount and end date of discount we cant give discount all days so problem was that in our database we did not have any columns like startdate and enddate theen it uses oown knowledege without looking into database so we have to solve this problem suing concept of "few shot learning" lets see how we will use it

In [78]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

q3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('17178.550000'),)]
Answer:17178.55
> Finished chain.


In [79]:
q4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('17482'),)]
Answer:17482
> Finished chain.


In [80]:
q5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(70,), (17,), (15,)]
Answer:70
> Finished chain.


Once again above is the wrong answer. We need to use SUM(stock_quantity). Let's run the query explicitly. We will use the result of this query later on in the notebook

In [81]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('102'),)]
Answer:102
> Finished chain.


### Few shot learning

We will use few shot learning to fix issues we have seen so far

In [82]:
few_shots = [
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': q3} ,
    
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : q4},
    
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : q5
     }
]

In [83]:
exmpl = {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : str(q5)
     }

# Embedding

In [84]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [85]:
embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [86]:
string = "How many white color Levi's shirt I have? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White',Result of the SQL query,q5"

In [87]:
emb = embeddings.embed_query(string)

In [88]:
emb

[0.0745023638010025,
 0.09563492238521576,
 0.043689947575330734,
 0.0679817795753479,
 -0.06574059277772903,
 0.09028568863868713,
 0.11574981361627579,
 0.028198814019560814,
 0.009002607315778732,
 -0.03358960524201393,
 0.03320828452706337,
 -0.05517894774675369,
 0.07388364523649216,
 -0.055601902306079865,
 -0.0139928562566638,
 0.08721467852592468,
 0.06056702509522438,
 -0.00888493936508894,
 -0.05119819566607475,
 -0.11206994950771332,
 -0.017493221908807755,
 -0.094902403652668,
 0.03700924292206764,
 0.02817426808178425,
 -0.006441052071750164,
 -0.009022473357617855,
 -0.03166380897164345,
 0.04303469881415367,
 0.04767058044672012,
 -0.032439589500427246,
 -0.10947806388139725,
 0.015154656022787094,
 0.02915545180439949,
 0.08769181370735168,
 -0.04680688679218292,
 -0.09288018941879272,
 -0.0011354589369148016,
 -0.05449717864394188,
 0.004341609310358763,
 -0.011137791909277439,
 0.0033972705714404583,
 -0.061633795499801636,
 -0.01183480117470026,
 0.04359811171889305,

In [89]:
len(emb),emb[:5]

(384,
 [0.0745023638010025,
  0.09563492238521576,
  0.043689947575330734,
  0.0679817795753479,
  -0.06574059277772903])

In [90]:
to_vectorize = [''.join(i.values()) for i in few_shots]

In [91]:
to_vectorize[2]

"How many white color Levi's shirt I have?SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'Result of the SQL query70"

# VectorStore

we have to store this embeding somewhere so to store this embding we have vectorStore so we can store aur embeding into this

the job of VectorStore is to take input questions convert into embedings and find the similar looking few shots examples

In [92]:
from langchain.vectorstores import Chroma

In [93]:
# !pip install chromadb==0.4.15

In [94]:
to_vectorize

["If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n Result of the SQL query17178.55",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'Result of the SQL query17482",
 "How many white color Levi's shirt I have?SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'Result of the SQL query70"]

In [95]:
vectorstore = Chroma.from_texts(to_vectorize, 
                                embedding= embeddings, 
                                metadatas= few_shots)

In [96]:
texts = ["Sample text 1", "Sample text 2", "Sample text 3"]
metadatas = [{"metadata_key": "metadata_value"} for _ in range(len(texts))]
ids = ['id1', 'id2', 'id3']

# Attempt to create the VectorStore using the simplified data
# vectorstore = Chroma.from_texts(texts=texts, metadatas=metadatas, ids=ids)


becouse of some issue i am not able to use perform this code so i check there documentation and get to know that they have made some changes 

## Tryning FAISS for VectorStore

In [97]:
from langchain.vectorstores import FAISS

In [98]:
vectorestore = FAISS.from_texts(to_vectorize, embeddings, few_shots, ids)

In [99]:
vectorestore

In [100]:
from langchain.prompts import SemanticSimilarityExampleSelector

In [101]:
example_selector = SemanticSimilarityExampleSelector(vectorstore= vectorestore, k=2)

this wont work for use coz we have used diffrent approch 

In [102]:
few_shots[1]

{'Question': 'If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?',
 'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
 'SQLResult': 'Result of the SQL query',
 'Answer': '17482'}

In [103]:
example_selector

SemanticSimilarityExampleSelector(vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x000002A710317950>, k=2, example_keys=None, input_keys=None)

In [104]:
example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Question': 'If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?',
  'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
  'SQLResult': 'Result of the SQL query',
  'Answer': '17482'},
 {'Question': "How many white color Levi's shirt I have?",
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
  'SQLResult': 'Result of the SQL query',
  'Answer': '70'}]

## Now we have to give prompt to llm do not use any of your genral knowledge and make things up instead only use columns from the given table and from given database

In [105]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [106]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [107]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


### so as you can see we langchain already provides sql prompt its really great

In [108]:
from langchain.prompts import PromptTemplate

In [109]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [110]:
from langchain.prompts import FewShotPromptTemplate

In [111]:
few = FewShotPromptTemplate(example_selector=example_selector, 
                      example_prompt=example_prompt,
                      suffix=PROMPT_SUFFIX, 
                      prefix=_mysql_prompt, input_variables=['input','table_info','top_k'] 
                     )

In [119]:
sql_db = SQLDatabaseChain.from_llm(llm, db, few)

In [124]:
sql_db("How many white color Levi's shirt I have?").get('result'),sql_db("How many white color Levi's shirt I have?")

('102',
 {'query': "How many white color Levi's shirt I have?", 'result': '102'})